In [1]:
import os
import functools

import cv2
import numpy as np

from tests.metaworld.envs.mujoco.sawyer_xyz.test_scripted_policies import ALL_ENVS, test_cases_latest_nonoise

/home/ykai/anaconda3/envs/metaGarage/lib/python3.9/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [2]:
def trajectory_generator(env, policy, act_noise_pct, res=(640, 480), camera='corner'):
    action_space_ptp = env.action_space.high - env.action_space.low

    env.reset()
    env.reset_model()
    o = env.reset()

    for _ in range(env.max_path_length):
        a = policy.get_action(o)
        a = np.random.normal(a, act_noise_pct * action_space_ptp)

        o, r, done, info = env.step(a)
        # Camera is one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
        # print(np.shape(env.sim.render(*res, mode='offscreen', camera_name=camera)[:,:,::-1]))
        yield r, done, info, env.sim.render(*res, mode='offscreen', camera_name=camera)[:,:,::-1]

# def writer_for(tag, fps, res):
#     if not os.path.exists('movies_my'):
#         os.mkdir('movies_my')
#     return cv2.VideoWriter(
#         f'movies_my/{tag}.avi',
#         cv2.VideoWriter_fourcc('M','J','P','G'),
#         fps,
#         res
#     )

def write_for_img(tag, img):
    if not os.path.exists('movies_my'):
        os.mkdir('movies_my')
    name = f'movies_my/{tag}.png'
    return cv2.imwrite(name, img)

In [3]:
!LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libGLEW.so:/usr/lib/nvidia/libGL.so


In [4]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/lib/nvidia
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/lib/nvidia-510
!export LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libGLEW.so

In [5]:
import os
os.getcwd()

'/ssd_2/ssd2048/papers/Sensory_extended_metaworld/metaworld/scripts'

In [6]:
resolution = (1920, 1080)
camera = 'topview' # one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
camera = ['topview', 'corner', 'corner2', 'corner3', 'behindGripper', 'gripperPOV']
my_camera = ['my_corner3', 'my_topview1']
camera = my_camera
flip=True # if True, flips output image 180 degrees

config = [
    # env, action noise pct, cycles, quit on success
    ('assembly-v2', np.zeros(4), 1, True),
]
for camera in camera:
    if camera in ['corner', 'corner2', 'corner3', 'gripperPOV', 'my_corner3']:
        flip = False
    else:
        flip = True
    for env, noise, cycles, quit_on_success in config:
        tag = env + '-noise-' + np.array2string(noise, precision=2, separator=',', suppress_small=True)\
            + '-cycles-'+ str(cycles) +'-camera-'+ camera

        policy = functools.reduce(lambda a,b : a if a[0] == env else b, test_cases_latest_nonoise)[1]
        env = ALL_ENVS[env]()
        env._partially_observable = False
        env._freeze_rand_vec = False
        env._set_task_called = True

        # writer = writer_for(tag, env.metadata['video.frames_per_second'], resolution)
        # for _ in range(cycles):
        #     for r, done, info, img in trajectory_generator(env, policy, noise, resolution, camera):
        #         if flip: img = cv2.rotate(img, cv2.ROTATE_180)
        #         writer.write(img)
        #         if quit_on_success and info['success']:
        #             break
        # writer.release()
        
        for _ in range(cycles):
            for r, done, info, img in trajectory_generator(env, policy, noise, resolution, camera):
                print(np.shape(img))
                if flip: img = cv2.rotate(img, cv2.ROTATE_180)
                write_for_img(tag, img)
                break
            
                if quit_on_success and info['success']:
                    break
        
    

/home/ykai/anaconda3/envs/metaGarage/lib/python3.9/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/ykai/anaconda3/envs/metaGarage/lib/python3.9/site-packages/metaworld/policies/policy.py:41: UserWarning: Constant(s) may be too high. Environments clip response to [-1, 1]
  warnings.warn('Constant(s) may be too high. Environments clip response to [-1, 1]')


Found 3 GPUs for rendering. Using device 0.
(1080, 1920, 3)
(1080, 1920, 3)


my_corner3 camera에러가 나올경우, pip install --user .로 현 경로의 metaworld 재설치.

In [8]:
import glob
import os
import sys

def get_egg_file(module_name):
    def f(packages):
        return glob.glob(
            os.path.join(os.path.dirname(os.path.dirname(sys.executable)),
                         'lib', 'python*', packages, module_name + '.egg-link'))

    return f('site-packages') or f('dist-packages')


egg_file = get_egg_file('metaworld')
print(egg_file)
# if egg_file:
#     os.remove(egg_file[0])

with open(egg_file[0], "r") as file:
    data = file.read()
    print(data)
    # file.write("Hello~ \n")
    # file.write("World!")

[]


IndexError: list index out of range

In [9]:
def my_trajectory_generator(env, policy, act_noise_pct, res=(640, 480), camera='corner', depth = True):
    action_space_ptp = env.action_space.high - env.action_space.low
    print("E1")
    env.reset()
    env.reset_model()
    o = env.reset()

    for _ in range(env.max_path_length):
        a = policy.get_action(o)
        a = np.random.normal(a, act_noise_pct * action_space_ptp)

        o, r, done, info = env.step(a)
        # Camera is one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
        # print(np.shape(env.sim.render(*res, mode='offscreen', camera_name=camera)[:,:,::-1]))
        img, d = env.sim.render(*res, mode='offscreen', camera_name=camera,depth = depth)
        # print(depth)
        print("E")
        yield r, done, info, img, d
        
def write_for_img(tag, img):
    if not os.path.exists('movies_my'):
        os.mkdir('movies_my')
    name = f'movies_my/{tag}.png'
    return cv2.imwrite(name, img)

In [22]:
import numpy as np
import cv2


def bright_img(tag, img):
    if not os.path.exists('movies_my'):
        os.mkdir('movies_my')
    name = f'movies_my/{tag}.png'
    src = img
    val = 100
    array = np.full(src.shape, (val, val, val), dtype=np.uint8)

    add = cv2.add(src, array)
    return cv2.imwrite(name, add)


In [26]:
resolution = (1920, 1080)
camera = 'topview' # one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
camera = ['topview', 'corner', 'corner2', 'corner3', 'behindGripper', 'gripperPOV']
my_camera = ['my_corner3', 'my_topview1']
camera = my_camera
flip=True # if True, flips output image 180 degrees

config = [
    # env, action noise pct, cycles, quit on success
    ('assembly-v2', np.zeros(4), 1, True),
]

for camera in camera:
    if camera in ['corner', 'corner2', 'corner3', 'gripperPOV', 'my_corner3']:
        flip = False
    else:
        flip = True
    for env, noise, cycles, quit_on_success in config:
        tag = env + '-noise-' + np.array2string(noise, precision=2, separator=',', suppress_small=True)\
            + '-cycles-'+ str(cycles) +'-camera-'+ camera

        policy = functools.reduce(lambda a,b : a if a[0] == env else b, test_cases_latest_nonoise)[1]
        env = ALL_ENVS[env]()
        env._partially_observable = False
        env._freeze_rand_vec = False
        env._set_task_called = True
        for _ in range(cycles):
            for r, done, info, img, depth in my_trajectory_generator(env, policy, noise, resolution, camera, depth = True):

                if flip: img = cv2.rotate(img, cv2.ROTATE_180)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                write_for_img(tag, img)
                print(depth)
                # d = d[:, np.newaxis] 
                # print(img.shape)
                # print(d.shape)
                # print(d*255)
                depth = (np.max(depth)-depth) / (np.max(depth) - np.min(depth))
                # depth = (depth-np.min(depth)) / (np.max(depth) - np.min(depth))
                depth = np.asarray(depth * 15, dtype=np.uint8)
                print(np.shape(depth))
                print(depth)
                write_for_img(tag+'_depth', depth)
                break
            
                if quit_on_success and info['success']:
                    break

E1
E
[[0.9997777  0.99977785 0.9997781  ... 0.9997956  0.9997954  0.99979514]
 [0.99977785 0.9997781  0.9997783  ... 0.9997958  0.99979556 0.9997953 ]
 [0.99977803 0.9997783  0.99977845 ... 0.999796   0.99979573 0.9997955 ]
 ...
 [0.9902057  0.9902049  0.9902041  ... 0.9886916  0.98869085 0.9886901 ]
 [0.9902004  0.9901996  0.99019885 ... 0.9886864  0.98868555 0.9886848 ]
 [0.99019516 0.9901944  0.99019355 ... 0.9886811  0.9886803  0.9886795 ]]
(1080, 1920)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [7 7 7 ... 8 8 8]
 [7 7 7 ... 8 8 8]
 [7 7 7 ... 8 8 8]]
E1
E
[[0.99442875 0.99442875 0.99442875 ... 0.99442875 0.99442875 0.99442875]
 [0.99442875 0.99442875 0.99442875 ... 0.99442875 0.99442875 0.99442875]
 [0.99442875 0.99442875 0.99442875 ... 0.99442875 0.99442875 0.99442875]
 ...
 [0.99442875 0.99442875 0.99442875 ... 0.99442875 0.99442875 0.99442875]
 [0.99442875 0.99442875 0.99442875 ... 0.99442875 0.99442875 0.99442875]
 [0.99442875 0.99442875 0.99442875 ... 0.99

In [12]:
img[:, :, 0], img[:, :, 2] = img[:, :, 2], img[:, :, 0] 

In [30]:

name = f'movies_my/linemod_depth2.png'

img = cv2.imread(name)
print(np.max(img))
print(np.min(img))


15
0


In [29]:
gray_img = cv2.imread(name)
print(gray_img.dtype)
print(np.shape(gray_img))

uint8
(480, 640, 3)
